## Метод решающего дерева и случайного леса

### Лабораторная работа №5

#### Задание 1
Провести классификацию найденного датасета, методами решающего дерева и случайного леса . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Метод решающего дерева:
Гиперпараметры:
   - max_depth: Максимальная глубина дерева. .
   - min_samples_split: Минимальное количество выборок, необходимых для разделения внутреннего узла.
   - min_samples_leaf: Минимальное количество выборок, необходимых для существования листового узла.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm

# Загрузка датасета
file_path = "Electric_Vehicle_Population_Data.csv"
df = pd.read_csv(file_path)
df.dropna(inplace=True)
label_encoder = LabelEncoder()
df['Electric Vehicle Type'] = label_encoder.fit_transform(df['Electric Vehicle Type'])
df = df.sample(n=10000)

# Разделение на признаки (X) и целевую переменную (y)
X = df[['Postal Code', 'Model Year']]
y = df['Electric Vehicle Type']
# Разделение данных на тренировочный и тестовый наборы
# random_state=42 - гарантирует, что данные каждый раз будут одинакого разбиваться
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков (нормализация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid_tree = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
# Создание модели решающего дерева
decision_tree = DecisionTreeClassifier()

# Поиск лучших параметров для решающего дерева
grid_search_tree = GridSearchCV(decision_tree, param_grid_tree, refit=True, verbose=0, cv=5)
grid_search_tree.fit(X_train_scaled, y_train)

# Лучшие параметры для решающего дерева
best_params_tree = grid_search_tree.best_params_
best_score_tree = grid_search_tree.best_score_

best_tree = DecisionTreeClassifier(**best_params_tree)
best_tree.fit(X_train_scaled, y_train)
accuracy_tree = best_tree.score(X_test_scaled, y_test)
print("Лучшие параметры для решающего дерева:", best_params_tree)
print("Точность решающего дерева на тестовом наборе:", accuracy_tree)

predictions = best_tree.predict(X_test_scaled)

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))

Лучшие параметры для решающего дерева: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10}
Точность решающего дерева на тестовом наборе: 0.782

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.79      0.97      0.87      1556
           1       0.54      0.11      0.19       444

    accuracy                           0.78      2000
   macro avg       0.67      0.54      0.53      2000
weighted avg       0.74      0.78      0.72      2000



## Метод случаного леса:

Гиперпараметры:
   - n_estimators: Количество деревьев в лесу.
   - max_depth: Максимальная глубина каждого дерева в лесу.
   - min_samples_split: Минимальное количество выборок, необходимых для разделения внутреннего узла дерева.
   - min_samples_leaf: Минимальное количество выборок, необходимых для существования листового узла.

In [3]:
# Определение параметров для случайного леса
# Указал маленькие значения для ускорения обучения
param_grid_forest = {
    'n_estimators': range(2, 10),
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4]
}
# Создание модели случайного леса
random_forest = RandomForestClassifier()

# Поиск лучших параметров для случайного леса
grid_search_forest = GridSearchCV(random_forest, param_grid_forest, refit=True, verbose=0, cv=5)
grid_search_forest.fit(X_train_scaled, y_train)

# Лучшие параметры для случайного леса
best_params_forest = grid_search_forest.best_params_
best_score_forest = grid_search_forest.best_score_
best_forest = RandomForestClassifier(**best_params_forest)
best_forest.fit(X_train_scaled, y_train)
accuracy_forest = best_forest.score(X_test_scaled, y_test)
print("Лучшие параметры для случайного леса:", best_params_forest)
print("Точность случайного леса на тестовом наборе:", accuracy_forest)

predictions = best_forest.predict(X_test_scaled)

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))

Лучшие параметры для случайного леса: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 5}
Точность случайного леса на тестовом наборе: 0.7805

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.78      0.99      0.88      1556
           1       0.59      0.04      0.07       444

    accuracy                           0.78      2000
   macro avg       0.68      0.52      0.47      2000
weighted avg       0.74      0.78      0.70      2000

